**INSTALLING REQUIRED MODULES**

>Pandas

In [ ]:
%pip install pandas

     ---------------------------------------- 10.6/10.6 MB 1.0 MB/s eta 0:00:00
     -------------------------------------- 502.3/502.3 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


>Image

In [ ]:
%pip install Image

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 8.0/8.0 MB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 41.2/41.2 kB 2.1 MB/s eta 0:00:00
     -------------------------------------- 341.8/341.8 kB 4.3 MB/s eta 0:00:00
  Running setup.py install for Image: started
  Running setup.py install for Image: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  DEPRECATION: Image is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


>Torch

In [ ]:
%pip install torch

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 1.2/1.2 MB 6.9 MB/s eta 0:00:00
     ---------------------------------------- 62.5/62.5 kB 3.3 MB/s eta 0:00:00
     -------------------------------------- 172.3/172.3 MB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 4.6 MB/s eta 0:00:00
     -------------------------------------- 133.1/133.1 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 96.7/96.7 kB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB 1.7 MB/s eta 0:00:00
     -------------------------------------- 123.2/123.2 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 157.0/157.0 kB 4.7 MB/s eta 0:00:00
     -------------------------------------- 536.2/536.2 kB 1.2 MB/s eta 0:00:00
  Using cached torch-2.0.1-cp311-cp311-win_amd64

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


>Matplotlib

In [ ]:
%pip install matplotlib


     ---------------------------------------- 7.6/7.6 MB 5.1 MB/s eta 0:00:00
     -------------------------------------- 163.0/163.0 kB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 55.4/55.4 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 98.3/98.3 kB 5.9 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


>Tensorflow

In [ ]:
%pip install tensorflow

  Using cached tensorflow-2.12.0-cp311-cp311-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp311-cp311-win_amd64.whl (272.9 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.8.0-cp311-cp311-win_amd64.whl (2.6 MB)
  Using cached jax-0.4.10-py3-none-any.whl
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached grpcio-1.54.2-cp311-cp311-win_amd64.whl (4.1 MB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached ml_dtypes-0.1.0-cp311-cp311-win_amd64.whl (120 kB)
  Using cached scipy-1.10.1-cp311-cp311-win_amd64.whl (42.2 MB)
  Using cached google_auth-2.18.0-py2.py3-none-any.whl (178 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached Markdo

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install tensorflow_hub

     ------------------------------------ 100.6/100.6 kB 579.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


>OpenCV

In [2]:
%pip install opencv-python

%pip install opencv-contrib-python

%pip install opencv-python opencv-python-headless

%python3 -m pip install opencv-python

%pip3 install opencv-python --upgrade

     ---------------------------------------- 38.2/38.2 MB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 14.8/14.8 MB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 44.9/44.9 MB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 38.1/38.1 MB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
UsageError: Line magic function `%python3` not found (But cell magic `%%python3` exists, did you mean that instead?).


# CODE STARTS HERE~

**VIDEO FRAME EXTRACTION FOR INPUT VIDEO**

In [6]:
import cv2
import os
count=0
success=True
video=cv2.VideoCapture('InputVideo.mp4')
if not video.isOpened():
    print("Error opening video file!")
directory = "InputFrames"
parent_dir = "E:\saintgits folder\T (Mini Project)\CODE"
path = os.path.join(parent_dir, directory)
os.mkdir(path)
while success:
    success,image=video.read()
    if success:
        cv2.imwrite("InputFrames/frame%d.jpg" % count,image)
        count+=1

**VIDEO FRAME EXTRACTION FOR VIDEOS IN THE DATABASE**

In [7]:
n=int(input("Enter the number of videos in the DataBase: "))

def split(x):
  count=0
  success=True
  video=cv2.VideoCapture('Database/Video%d.mp4' % x)
  if not video.isOpened():
    print("Error opening database video file : Video%d!" % x)
  print("Currently Processing -> Video",x,": \n")

  directory = "Temp%d" % x
  parent_dir = "E:\saintgits folder\T (Mini Project)\CODE"
  path = os.path.join(parent_dir, directory)
  os.mkdir(path)

  while success:
    success,image=video.read()
    if success:
      cv2.imwrite("E:\saintgits folder\T (Mini Project)\CODE\Temp%d/frame%d.jpg" % (x,count),image)
      count+=1
    else:
      break

for i in range(n):
  split(i)

Currently Processing -> Video 0 : 

Currently Processing -> Video 1 : 

Currently Processing -> Video 2 : 

Currently Processing -> Video 3 : 



**RESNET AND COSINE SIMILARITY MEASURE FOR INPUT VIDEO FRAMES AND VIDEOS IN THE DATABASE**

In [9]:
import numpy as np
import os
import pandas as pd
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms

resnet18 = models.resnet18(pretrained=True)

feature_extractor = torch.nn.Sequential(*list(resnet18.children())[:-1])

def extract_features(image_path, feature_extractor):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
    image = transform(image).unsqueeze(0)

    with torch.no_grad():
        features = feature_extractor(image).squeeze()

    return features.numpy()

def compare_images(image1_path, image2_path, feature_extractor):
    features1 = extract_features(image1_path, feature_extractor)
    features2 = extract_features(image2_path, feature_extractor)

    similarity = np.dot(features1, features2) / (np.linalg.norm(features1) * np.linalg.norm(features2))
    return similarity

>Reading the Cosine Similarities into a .csv file

In [16]:
image1_path = 'E:\saintgits folder\T (Mini Project)\CODE\InputFrames\\frame61.jpg'

for v in range(4):
    print("\n\nVideo%d:\n\n" % v)

    dir_path = r'E:\saintgits folder\\T (Mini Project)\\CODE\\Temp%d' % v
    n=(len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]))
    
    L=[]

    print("Done With:\n")
    
    for i in range(n):
        print(i)
        image2_path='E:\saintgits folder\T (Mini Project)\CODE\Temp%d\\frame%d.jpg' % (v,i)
        L.append([v,compare_images(image1_path, image2_path, feature_extractor)])

    df = pd.DataFrame(L,columns=['Video','Similarity'])
    if v==0:
        df.to_csv('CosSimData.csv',index=False)
    else:
        df.to_csv('CosSimData.csv', mode='a', index=False, header=False)



Video0:


Done With:

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
27

___

In [23]:
df=pd.read_csv('CosSimData.csv')
print(df.loc[df['Similarity'] == df['Similarity'].max()])

      Video  Similarity
3825      2    0.997539


In [1]:
%pip install sklearn

  Using cached sklearn-0.0.post5.tar.gz (3.7 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2968 sha256=c21f0acb764af350dda7213970a6237196b6003ee3dab270e2c8cb769fcae39f
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\0f\80\97\b34b7fe3ca2f4da52ac353b60182a7a00fff534bb20bf2abd7
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install --upgrade pip

     ---------------------------------------- 2.1/2.1 MB 966.8 kB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [9]:
%pip install scikit-learn

                                              0.0/8.3 MB ? eta -:--:--
                                              0.0/8.3 MB ? eta -:--:--
                                              0.0/8.3 MB ? eta -:--:--
                                              0.1/8.3 MB 544.7 kB/s eta 0:00:16
                                              0.1/8.3 MB 544.7 kB/s eta 0:00:16
                                              0.1/8.3 MB 607.9 kB/s eta 0:00:14
                                              0.1/8.3 MB 607.9 kB/s eta 0:00:14
     -                                        0.2/8.3 MB 838.1 kB/s eta 0:00:10
     -                                        0.4/8.3 MB 1.1 MB/s eta 0:00:08
     -                                        0.4/8.3 MB 1.1 MB/s eta 0:00:08
     --                                       0.5/8.3 MB 1.1 MB/s eta 0:00:07
     --                                       0.5/8.3 MB 1.1 MB/s eta 0:00:07
     --                                       0.5/8.3 MB 962.6 kB/s eta 0:0

In [1]:
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to extract color histograms as features from a video
def extract_color_histograms(video_path, bins=(8, 8, 8)):
    cap = cv2.VideoCapture(video_path)
    features = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        features.append(hist)
    cap.release()
    return np.array(features)

# Function to calculate video similarity using color histograms
def calculate_video_similarity(video1_path, video2_path):
    vf1 = extract_color_histograms(video1_path)
    vf2 = extract_color_histograms(video2_path)
    similarity_matrix = cosine_similarity(vf1, vf2)
    average_similarity = np.mean(similarity_matrix)
    return average_similarity

# Example usage: check similarity between two videos
video1_path = 'E:\saintgits folder\T (Mini Project)\CODE\InputVideo.mp4'
video2_path = 'E:\saintgits folder\T (Mini Project)\CODE\Database\Video0.mp4'
similarity_score = calculate_video_similarity(video1_path, video2_path)
print("Video similarity score:", similarity_score)


Video similarity score: 0.45822707


In [3]:
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to extract color histograms as features from a video
def extract_color_histograms(video_path, bins=(8, 8, 8)):
    cap = cv2.VideoCapture(video_path)
    features = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        features.append(hist)
    cap.release()
    return np.array(features)

# Function to calculate video similarity using color histograms
def calculate_video_similarity(video1_path, video2_path):
    vf1 = extract_color_histograms(video1_path)
    vf2 = extract_color_histograms(video2_path)
    similarity_matrix = cosine_similarity(vf1, vf2)
    average_similarity = np.mean(similarity_matrix)
    return average_similarity

# Example usage: check similarity between two videos
video1_path = 'E:\saintgits folder\T (Mini Project)\CODE\InputVideo.mp4'

number=int(input("Enter the number of videos in the DB: "))

for i in range(number):
    print(i,":\t")
    video2_path = 'E:\saintgits folder\T (Mini Project)\CODE\Database\Video%d.mp4' % i
    similarity_score = calculate_video_similarity(video1_path, video2_path)
    print("Video similarity score:", similarity_score)


0 :	
Video similarity score: 0.45822707
1 :	
Video similarity score: 0.06255585
2 :	
Video similarity score: 0.061472297
3 :	
Video similarity score: 0.31092092


In [5]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Function to calculate SSIM between two videos
def calculate_ssim(video1, video2):
    cap1 = cv2.VideoCapture(video1)
    cap2 = cv2.VideoCapture(video2)

    ssim_scores = []

    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        if not ret1 or not ret2:
            break

        # Convert frames to grayscale
        frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Calculate SSIM between frames
        score = ssim(frame1_gray, frame2_gray)

        ssim_scores.append(score)

    cap1.release()
    cap2.release()

    # Calculate average SSIM score
    avg_ssim = np.mean(ssim_scores)

    return avg_ssim

# Example usage
video1 = "E:\saintgits folder\T (Mini Project)\CODE\InputVideo.mp4"

number=int(input("Enter the number of videos in the DB: "))

for i in range(number):
    print(i,":\t")
    video2= 'E:\saintgits folder\T (Mini Project)\CODE\Database\Video%d.mp4' % i
    similarity_score = calculate_ssim(video1, video2)
    print("Similarity score:", similarity_score)


0 :	
Similarity score: 0.26838284823082464
1 :	


ValueError: Input images must have the same dimensions.

In [8]:
%pip install faiss-cpu

                                              0.0/10.8 MB ? eta -:--:--
                                              0.0/10.8 MB 1.4 MB/s eta 0:00:08
                                              0.1/10.8 MB 1.1 MB/s eta 0:00:11
                                              0.1/10.8 MB 1.1 MB/s eta 0:00:11
                                             0.2/10.8 MB 985.7 kB/s eta 0:00:11
                                             0.2/10.8 MB 985.7 kB/s eta 0:00:11
                                             0.2/10.8 MB 985.7 kB/s eta 0:00:11
                                             0.2/10.8 MB 985.7 kB/s eta 0:00:11
                                             0.2/10.8 MB 985.7 kB/s eta 0:00:11
                                             0.2/10.8 MB 985.7 kB/s eta 0:00:11
                                             0.2/10.8 MB 985.7 kB/s eta 0:00:11
                                             0.2/10.8 MB 402.0 kB/s eta 0:00:27
     -                                       0.3/10

In [7]:
import numpy as np
import faiss  # for LSH indexing

# Step 1: Feature extraction with CNN
import numpy as np
import os
import pandas as pd
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms

resnet18 = models.resnet18(pretrained=True)

feature_extractor = torch.nn.Sequential(*list(resnet18.children())[:-1])

def extract_features(image_path, feature_extractor):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
    image = transform(image).unsqueeze(0)

    with torch.no_grad():
        features = feature_extractor(image).squeeze()

    return features.numpy()

# Step 2: Video representation
n=int(input("Enter the number of videos in DB:"))
video_rep=[]
for i in range(n):
    print("Video: ",i)
    dir_path = r'E:\\saintgits folder\\T (Mini Project)\\CODE\\Temp%d' % i
    k=(len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]))
    frame_features=[]
    for j in range(k):
        image_path='E:\\saintgits folder\\T (Mini Project)\\CODE\\Temp%d\\frame%d.jpg' % (i,j)
        frame_features.append(extract_features(image_path, feature_extractor)) # Shape: (num_frames, feature_dim)
        # Perform average pooling over frames
    video_rep=np.mean(frame_features, axis=0)
    if i==0:
        # Step 3: Indexing with LSH
        # Initialize LSH index
        index = faiss.IndexLSH(2,len(video_rep))  # d is the dimensionality of the video representation

    # Add video representations to the index
    index.add(video_rep)

print("Done with video_rep")

c:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Video:  0


ValueError: not enough values to unpack (expected 2, got 1)

In [6]:
print(video_rep)

[[0, array([0.90570563, 0.9329373 , 0.9314423 , 1.0605346 , 0.9327587 ,
       0.8767069 , 0.93649185, 1.1649313 , 0.94375193, 0.9414775 ,
       0.8945305 , 0.8575462 , 0.9376407 , 0.91680723, 0.9145728 ,
       0.9579196 , 0.91070485, 1.3860754 , 0.9298172 , 0.85208124,
       0.91257715, 1.0660222 , 0.9133886 , 0.9757683 , 0.937677  ,
       0.9397352 , 0.97210974, 0.98650575, 0.88703376, 0.92262954,
       0.9268761 , 0.878517  , 0.8950737 , 0.87802386, 0.8874386 ,
       0.88737506, 0.95736164, 0.9554211 , 0.8680245 , 0.90043926,
       0.8977303 , 0.8833285 , 0.8588177 , 0.9574728 , 0.8551061 ,
       0.8892399 , 0.9259633 , 1.1847341 , 0.95625806, 0.880476  ,
       0.97027683, 0.9509857 , 0.87056243, 0.9111433 , 0.94513214,
       0.92930686, 1.0958382 , 0.9674151 , 1.0282089 , 0.94884515,
       0.9019754 , 0.9646608 , 0.97743607, 0.910972  , 0.877874  ,
       1.0446875 , 0.8637446 , 0.9244988 , 1.0314405 , 0.8731328 ,
       0.90437365, 0.8308137 , 0.8991855 , 0.88585085, 0.

In [5]:


# Step 4: Query processing
# Extract features from query video
query_video='E:\saintgits folder\T (Mini Project)\CODE\InputVideo.mp4'
query_features = extract_features(query_video)

# Search for similar videos
k = 10  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_features, k)

print(indices)
# Retrieve similar video paths from the indices
#similar_videos = database_videos[indices[0]]

# Print the paths of similar videos
"""for video_path in similar_videos:
    print(video_path)"""


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (4, 2) + inhomogeneous part.